<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#damage_zones(-)-to-calculate-parameters-of-damage-zones" data-toc-modified-id="damage_zones(-)-to-calculate-parameters-of-damage-zones-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>damage_zones( ) to calculate parameters of damage zones</a></span></li><li><span><a href="#impact_risk(-)-uncertainty-analysis" data-toc-modified-id="impact_risk(-)-uncertainty-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>impact_risk( ) uncertainty analysis</a></span></li></ul></div>

To install the module and any pre-requisites, from the base directory run:
```
pip install -r requirements.txt
pip install -e .
```

To download the postcode data required:

```
python download_data.py
```

In [1]:
from armageddon.damage import damage_zones, impact_risk
from armageddon.solver import Planet
import armageddon
import folium

### damage_zones( ) to calculate parameters of damage zones

Calculate the latitude and longitude of the surface zero location and the
list of airblast damage radii (m) for a given impact scenario.


__Parameters__

outcome: Dict

    the outcome dictionary from an impact scenario
        
lat: float

    latitude of the meteoroid entry point (degrees)

lon: float

    longitude of the meteoroid entry point (degrees)
    
bearing: float

    Bearing (azimuth) relative to north of meteoroid trajectory (degrees)
    
pressures: float, arraylike

    List of threshold pressures to define airblast damage levels
    
plot: bool

    Boolean value to decide plotting

__Return__

blat: float

    latitude of the surface zero point (degrees)
        
blon: float

    longitude of the surface zero point (degrees)
        
damrad: arraylike, float

    List of distances specifying the blast radii for the input damage levels
        
plot: plot object

    The plot specifying the areas effected by each damage level


__Examples__
    
    >>> import armageddon
    >>> outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
                   'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
                   'outcome': 'Airburst'}
    >>> armageddon.damage_zones(outcome, 52.79, -2.95, 135,
                                pressures=[1e3, 3.5e3, 27e3, 43e3])\


In [2]:
outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
           'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
           'outcome': 'Airburst'}
arr = damage_zones(outcome, 52.79, -2.95, 135, 
                   pressures=[1e3, 3.5e3, 27e3, 43e3],
                   )
arr

(51.98371949678459,
 -2.8338706741653983,
 [115971.31673025587,
  42628.36651535611,
  9575.214234120964,
  5835.9834520793875])

_Explaination:_

damage_zones( ) returned a list of parameters with :

    surface zero latitude in degree:  51.98371949678459

    surface zero longitude in degree:  -2.8338706741653983

    blast radii with the pressure level of 1e3:  115971.31673025587

    blast radii with the pressure level of 3.5e3:  42628.36651535611

    blast radii with the pressure level of 27e3:  9575.214234120964

    blast radii with the pressure level of 43e3:  5835.9834520793875


In [3]:
a = armageddon.plot_circle(arr[0], arr[1], arr[2][0])
armageddon.plot_circle(arr[0], arr[1], arr[2][1], map = a)
armageddon.plot_circle(arr[0], arr[1], arr[2][2], map = a)
armageddon.plot_circle(arr[0], arr[1], arr[2][3], map = a)

_Explaination:_
    
    The damage zones are marked on the plot above. As the plot and calculations shows, when the radius(horizontal range) goes up, the pressure in this wave decreases. When the asteroids blast occurs, the nearer area will experience relatively greater pressure, while the further area will experience relatively lower pressure.This is also proved by the equation:
    
\begin{equation*}
p(r) = 3.14 \times 10^{11} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-1.3} + 1.8 \times 10^{7} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-0.565}
\end{equation*}

$p$ (in Pa) (above ambient, also known as overpressure): the pressure in this wave

$E_k$ (in kilotons of TNT equivalent): explosion energy

$z_b$ (in m): burst altitude

$r$ (in m): horizontal range


### impact_risk( ) uncertainty analysis

Perform an uncertainty analysis to calculate the risk for each affected UK postcode or postcode sector


__Parameters__

planet: armageddon.Planet instance

    The Planet instance from which to solve the atmospheric entry

means: dict

    A dictionary of mean input values for the uncertainty analysis. This should include values for ``radius``, ``angle``, ``strength``, ``density``, ``velocity``, ``lat``, ``lon`` and ``bearing``

stdevs: dict

    A dictionary of standard deviations for each input value. This should include values for ``radius``, ``angle``, ``strength``, ``density``, ``velocity``, ``lat``, ``lon`` and ``bearing``

pressure: float

    A single pressure at which to calculate the damage zone for each impact

nsamples: int

    The number of iterations to perform in the uncertainty analysis

sector: logical, optional

    If True (default) calculate the risk for postcode sectors, otherwise calculate the risk for postcodes

__Return__

risk: DataFrame

    A pandas DataFrame with columns for postcode (or postcode sector) and the associated risk. These should be called ``postcode`` or ``sector``, and ``risk``.


In [4]:
p = Planet()
fiducial_means = {'radius': 35, 'angle': 45, 'strength': 1e7,
                  'density': 3000, 'velocity': 19e3,
                  'lat': 53.0, 'lon': -2.5, 'bearing': 115.}
fiducial_stdevs = {'radius': 1, 'angle': 1, 'strength': 5e6,
                   'density': 500, 'velocity': 1e3,
                   'lat': 0.025, 'lon': 0.025, 'bearing': 0.5}
df = impact_risk(p, means=fiducial_means, stdevs=fiducial_stdevs, pressure=27.e3, nsamples=10, sector=True)
df.head(10)

,postcode sector,risk
0,NG9 2,6706.0
1,NG3 5,4666.0
2,NG1 6,140.5
3,NG111,0.0
4,NG3 4,2061.5
5,NG8 3,4902.0
6,NG9 1,5026.0
7,NG7 5,3923.0
8,NG808,0.0
9,NG3 7,3654.0


_Explaination:_

    The above example calculates the risk parameter of each sector in England caused by the asteroid's blast, which predicts the population of each area that could be affected. It takes parameters of the planet's condition and returns the postcodes of the area that could be affected. A number of tests are carried out to calculate the probability of the area being affected. The production of the probability and the population of the area is the risk number of the area.